02-regression Home Work

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
!python -m wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip -o bank_marketing.zip


Saved under bank_marketing (3).zip


In [3]:
df = pd.read_csv('bank-full.csv', sep=';')

In [4]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
features = ['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays','previous', 'poutcome', 'y']

In [6]:
df = df[features]

In [7]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [9]:
df['education'].mode()

0    secondary
Name: education, dtype: object

In [12]:
df[['age','balance','day','duration','campaign','pdays','previous']].corr()

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


In [13]:
df.y = (df['y']=='yes').astype(int)

In [14]:
df.y

0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int32

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [17]:
df_train, df_val = train_test_split(df_train_full, test_size=0.2, random_state=42)

In [19]:
df_train.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
25560,36,management,married,tertiary,-485,yes,cellular,19,nov,278,1,-1,0,unknown,0
19774,55,retired,married,primary,2325,yes,telephone,8,aug,87,7,-1,0,unknown,0
35954,42,admin.,married,secondary,3104,no,cellular,8,may,144,2,-1,0,unknown,0
36483,36,blue-collar,married,secondary,-185,yes,cellular,12,may,167,2,350,2,failure,0
1260,60,retired,married,primary,-2,yes,unknown,8,may,75,1,-1,0,unknown,0


In [20]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

del df_train['y']
del df_val['y']
del df_test['y']

In [26]:
categorical = ['education','housing','contact','poutcome']

In [22]:
from sklearn.metrics import mutual_info_score

In [24]:
def calculate_mi(series): 
 return mutual_info_score(series, df_train_full.y) 

In [27]:
df_mi = df_train_full[categorical].apply(calculate_mi) 
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI') 
df_mi

,MI
poutcome,0.029257
contact,0.014164
housing,0.009800
education,0.002458


In [28]:
train_dict = df_train.to_dict(orient='records')

In [29]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [30]:
X_train = dv.transform(train_dict)

In [31]:
X_train[0]

array([  36., -485.,    1.,    1.,    0.,    0.,   19.,  278.,    0.,
          0.,    1.,    0.,    0.,    1.,    0.,    0.,    0.,    0.,
          1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    1.,    0.,    0.,   -1.,    0.,    0.,    0.,
          1.,    0.])

In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [36]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [37]:
val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [38]:
y_pred = model.predict_proba(X_val)[:, 1]

In [39]:
y = y_pred >= 0.5

In [40]:
accuracy = (y_val == y).mean()
accuracy

0.901022947193807

In [45]:
features_wo_age = ['job', 'marital', 'education', 'balance', 'housing', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays','previous', 'poutcome']

In [46]:
train_dict_wo_age = df_train[features_wo_age].to_dict(orient='records')
train_dict_wo_age
X_train_wo_age = dv.transform(train_dict_wo_age)
model.fit(X_train_wo_age, y_train)

X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)[:, 1]
y = y_pred >= 0.5
accuracy_wo_age = (y_val == y).mean()
accuracy_wo_age

0.9008847110865358

In [47]:
age = accuracy - accuracy_wo_age
age

0.00013823610727126123

In [48]:
features_wo_balance = ['age', 'job', 'marital', 'education', 'housing', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays','previous', 'poutcome']
train_dict_wo_balance = df_train[features_wo_balance].to_dict(orient='records')
train_dict_wo_balance
X_train_wo_balance = dv.transform(train_dict_wo_balance)
model.fit(X_train_wo_balance, y_train)

X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)[:, 1]
y = y_pred >= 0.5
accuracy_wo_balance = (y_val == y).mean()
accuracy - accuracy_wo_balance

-0.00027647221454241144

In [49]:
features_wo_marital = ['age', 'job', 'education', 'balance', 'housing', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays','previous', 'poutcome']
train_dict_wo_marital = df_train[features_wo_marital].to_dict(orient='records')
train_dict_wo_marital
X_train_wo_marital = dv.transform(train_dict_wo_marital)
model.fit(X_train_wo_marital, y_train)

X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)[:, 1]
y = y_pred >= 0.5
accuracy_wo_marital = (y_val == y).mean()
accuracy - accuracy_wo_marital

-0.0008294166436273454

In [55]:
features_wo_previous = ['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'poutcome']
train_dict_wo_previous = df_train[features_wo_previous].to_dict(orient='records')
train_dict_wo_previous
X_train_wo_previous = dv.transform(train_dict_wo_previous)
model.fit(X_train_wo_previous, y_train)

X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)[:, 1]
y = y_pred >= 0.5
accuracy_wo_previous = (y_val == y).mean()
accuracy - accuracy_wo_previous

-0.00027647221454241144

In [57]:
-0.00027647221454241144 < -0.00027647221454241144

False

In [58]:
for c in [0.01, 0.1, 1, 10, 100]:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train,y_train)
    val_dict = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dict)
    y_pred = model.predict_proba(X_val)[:, 1]
    y = y_pred >= 0.5
    accuracy = (y_val == y).mean()

    print(c, round(accuracy,3))

0.01 0.899
0.1 0.901
1 0.901
10 0.901
100 0.901
